In [4]:
import json
import pandas as pd


WIKI_JSON_PATH = './wikisql.json'
WIKI_SCHEMA_PATH = './wikisql-schema.csv' 

with open(WIKI_JSON_PATH) as f:
  wiki_json = json.load(f)

wiki_schema = pd.read_csv('wikisql-schema.csv', error_bad_lines=False)
wiki_schema.columns = [col.strip() for col in wiki_schema.columns]

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version.


  exec(code_obj, self.user_global_ns, self.user_ns)
b'Skipping line 334: expected 6 fields, saw 7\nSkipping line 437: expected 6 fields, saw 8\nSkipping line 443: expected 6 fields, saw 8\nSkipping line 737: expected 6 fields, saw 7\nSkipping line 738: expected 6 fields, saw 7\nSkipping line 1293: expected 6 fields, saw 7\nSkipping line 1294: expected 6 fields, saw 7\nSkipping line 1295: expected 6 fields, saw 7\nSkipping line 1361: expected 6 fields, saw 7\nSkipping line 1362: expected 6 fields, saw 7\nSkipping line 1363: expected 6 fields, saw 7\nSkipping line 1364: expected 6 fields, saw 7\nSkipping line 1827: expected 6 fields, saw 7\nSkipping line 1959: expected 6 fields, saw 7\nSkipping line 1960: expected 6 fields, saw 8\nSkipping line 3288: expected 6 fields, saw 7\nSkipping line 3396: expected 6

In [5]:
wiki_json[0]

{'query-split': 'N/A',
 'sentences': [{'original': 'Tell me what the notes are for South Australia ',
   'question-split': 'train',
   'table-id': '1-1000181-1',
   'text': 'Tell me what the notes are for South Australia',
   'variables': {'var0': 'SOUTH AUSTRALIA'}}],
 'sql': ['SELECT TABLEalias0.NOTES_FIELD FROM TABLE AS TABLEalias0 WHERE TABLEalias0.CURRENT_SLOGAN_FIELD = "var0" ;'],
 'sql-original': ['SELECT  col5 FROM table WHERE col3 = SOUTH AUSTRALIA'],
 'variables': [{'example': 'SOUTH AUSTRALIA',
   'location': 'both',
   'name': 'var0',
   'type': 'unknown'}]}

In [ ]:
wiki_schema

### Helper function

In [ ]:
def get_columns_by_table(id):
  columns =  wiki_schema[wiki_schema['Database name'] == id]['Field Name'].tolist()
  columns = [col.strip() for col in columns]
  return columns

def get_dtype_by_table(id):
  dtypes =  wiki_schema[wiki_schema['Database name'] == id]['Type'].tolist()
  dtypes = [dtype.strip() for dtype in dtypes]
  return dtypes

def get_schema_by_table(id):
  columns = get_columns_by_table(id)
  dtypes = get_dtype_by_table(id)

  schema = dict(zip(columns , dtypes))
  schema = ", ".join([f"{i} : {j}_TYPE" for (i,j) in schema.items()])
  return schema


get_schema_by_table('1-1000181-1')


In [ ]:
import re
def replace_col_alias(sql, table_id):
  col_alias = re.findall('col\d',sql)
  columns = get_columns_by_table(table_id)

  for alias in col_alias:
    sql = sql.replace(alias , columns[int(alias[-1])])
  return sql

def variable_rematch(sql, variables):
  values = list(variables.values())
  for var in values:
    try:
      int(var)
    except:
      sql = sql.replace(var , f"'{var}'")
  
  return sql
  

In [ ]:
def make_tokens(schema, question, sql):
  return "\n<|startoftex|>\n" + schema + "|" + question + "|" + sql + "\n<|endoftext|>\n"

In [ ]:
from pprint import pprint
from tqdm import tqdm

In [ ]:
json_data = []

for sample in tqdm(wiki_json):
  sentences = sample['sentences'][0]
  question = sentences['text']

  id = sentences['table-id']
  vars = sentences['variables']
  sql = sample['sql-original'][0]
  try:
    sql = replace_col_alias(sql, id)
    sql = variable_rematch(sql , vars)
    schema = get_schema_by_table(id)

    json_data.append({
        'schema' : schema,
        'question' : question,
        'sql' : sql
    })
  except:
    pass
  break

100%|██████████| 51159/51159 [31:24<00:00, 27.14it/s]


In [ ]:
with open('./wiki_clean.json', 'w') as f:
  json.dump(json_data,f)